<a href="https://colab.research.google.com/github/Jackelyneg/covid-19-image-classification/blob/OG/finalmap3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import folium
from math import cos, asin, sqrt, pi

import csv
from pandas import read_csv

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
data=pd.read_csv("/content/zip_code_database.csv")


In [6]:
decommissioned=data['decommissioned']
decommissioned.value_counts()

0    41684
1     1040
Name: decommissioned, dtype: int64

In [7]:
index_names=data[data["decommissioned"]==1].index


In [8]:
data.drop(index_names, inplace=True)

In [9]:
data.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population
0,501,UNIQUE,0,Holtsville,NaN,Internal Revenue Service,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Internal Revenue Service,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787,939",NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787,939",NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787,NaN,US,18.43,-67.15,0


In [10]:
data=(data[["zip", "primary_city", "state", "county", "latitude", "longitude"]])

In [51]:
data['zip'] = data['zip'].apply(lambda x: '{0:0>5}'.format(x))

In [52]:
data.head()

,zip,primary_city,state,county,latitude,longitude
0,00501,Holtsville,NY,Suffolk County,40.81,-73.04
1,00544,Holtsville,NY,Suffolk County,40.81,-73.04
2,00601,Adjuntas,PR,Adjuntas Municipio,18.16,-66.72
3,00602,Aguada,PR,Aguada Municipio,18.38,-67.18
4,00603,Aguadilla,PR,Aguadilla Municipio,18.43,-67.15


In [13]:
test_zip='07110'
test_zip

'07110'

In [14]:
for i, row in data.iterrows():
   
    if test_zip == row['zip']:
        lat2=row['latitude']
        long2=row['longitude']

print(f"Latitude: {lat2}")
print(f"Longitude: {long2}")

Latitude: 40.81
Longitude: -74.15


In [53]:
p=pi/180
results=[]
zips = []
lats = []
longs = []
for i, row in data.iterrows():
    lat1=row['latitude']
    long1=row['longitude']
    zip_code=row['zip']
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((long2-long1)*p))/2
    result=12742*asin(sqrt(a))
    results.append(result)
    zips.append(zip_code)
    lats.append(lat1)
    longs.append(long1)
   

In [18]:
df = pd.DataFrame(list(zip(results, zips, lats, longs)),
               columns =['distance', 'zip_codes', 'latitude', 'longitude'])
df

,distance,zip_codes,latitude,longitude
0,93.418449,00501,40.81,-73.04
1,93.418449,00544,40.81,-73.04
2,2616.659110,00601,18.16,-66.72
3,2581.292147,00602,18.38,-67.18
4,2576.626131,00603,18.43,-67.15
...,...,...,...,...
41679,4411.100748,99926,55.14,-131.49
41680,4533.074129,99927,56.30,-133.57
41681,4428.028540,99928,55.45,-131.79
41682,4411.868203,99929,56.41,-131.61


In [19]:
five_miles = df[df['distance']<= 5]
five_miles

,distance,zip_codes,latitude,longitude
2463,2.524840,07003,40.81,-74.18
2472,4.526675,07012,40.85,-74.16
2474,2.377777,07014,40.83,-74.14
2487,4.352806,07028,40.80,-74.20
2490,3.364887,07031,40.79,-74.12
2523,4.208066,07070,40.81,-74.10
2524,3.545581,07071,40.80,-74.11
2552,4.755824,07104,40.77,-74.17
2557,2.377867,07109,40.79,-74.16
2558,0.000000,07110,40.81,-74.15


In [49]:
#Base Map
usa = [37.090240, -95.712891]
Map = folium.Map(usa, zoom_start=4)
Map

In [50]:
for i in range(0, len(five_miles)):
    folium.Marker([five_miles.iloc[i]["latitude"], five_miles.iloc[i]["longitude"]], popup=five_miles.iloc[i]["zip_codes"]).add_to(Map)
Map

In [24]:
hosp_df=pd.read_csv('/content/hospital_zip.csv')
hosp_df.head()

,Unnamed: 0,Unnamed: 0.1,state,hospital_name,address,city,zip,total_beds_7_day_avg,inpatient_beds_used_7_day_avg,total_icu_beds_7_day_avg,icu_beds_used_7_day_avg,LAT,LNG
0,0,0,MA,ST VINCENT HOSPITAL,123 SUMMER STREET,WORCESTER,1608,243.0,168.6,28.0,21.7,42.261980,-71.801462
1,1,1,PA,CURAHEALTH PITTSBURGH,7777 STEUBENVILLE PIKE,OAKDALE,15071,15.0,12.6,0.0,0.0,40.412105,-80.187799
2,2,2,SD,MONUMENT HEALTH CUSTER HOSPITAL,1220 MONTGOMERY STREET,CUSTER,57730,11.0,6.7,0.0,0.0,43.720822,-103.641848
3,3,3,FL,ASCENSION ST VINCENT'S SOUTHSIDE,4201 BELFORT RD,JACKSONVILLE,32216,277.0,152.6,15.0,26.1,30.277467,-81.582904
4,4,4,KY,LOURDES HOSPITAL,1530 LONE OAK ROAD,PADUCAH,42003,358.0,117.1,23.6,23.4,37.004848,-88.577941


In [54]:
hosp_df=(hosp_df[["hospital_name", "address", "city", "state", "zip", "LAT", "LNG"]])
hosp_df.head()

,hospital_name,address,city,state,zip,LAT,LNG
0,ST VINCENT HOSPITAL,123 SUMMER STREET,WORCESTER,MA,01608,42.261980,-71.801462
1,CURAHEALTH PITTSBURGH,7777 STEUBENVILLE PIKE,OAKDALE,PA,15071,40.412105,-80.187799
2,MONUMENT HEALTH CUSTER HOSPITAL,1220 MONTGOMERY STREET,CUSTER,SD,57730,43.720822,-103.641848
3,ASCENSION ST VINCENT'S SOUTHSIDE,4201 BELFORT RD,JACKSONVILLE,FL,32216,30.277467,-81.582904
4,LOURDES HOSPITAL,1530 LONE OAK ROAD,PADUCAH,KY,42003,37.004848,-88.577941


In [26]:
hosp_df['zip'] = hosp_df['zip'].apply(lambda x: '{0:0>5}'.format(x))
hosp_df

,hospital_name,address,city,state,zip,LAT,LNG
0,ST VINCENT HOSPITAL,123 SUMMER STREET,WORCESTER,MA,01608,42.261980,-71.801462
1,CURAHEALTH PITTSBURGH,7777 STEUBENVILLE PIKE,OAKDALE,PA,15071,40.412105,-80.187799
2,MONUMENT HEALTH CUSTER HOSPITAL,1220 MONTGOMERY STREET,CUSTER,SD,57730,43.720822,-103.641848
3,ASCENSION ST VINCENT'S SOUTHSIDE,4201 BELFORT RD,JACKSONVILLE,FL,32216,30.277467,-81.582904
4,LOURDES HOSPITAL,1530 LONE OAK ROAD,PADUCAH,KY,42003,37.004848,-88.577941
...,...,...,...,...,...,...,...
1995,MEMORIAL HOSPITAL OF GARDENA,1145 W REDONDO BEACH BLVD,GARDENA,CA,90247,33.890853,-118.297967
1996,MCLAREN PORT HURON,1221 PINE GROVE AVE,PORT HURON,MI,48060,42.980388,-82.461014
1997,TRI-CITY MEDICAL CENTER,4002 VISTA WAY,OCEANSIDE,CA,92056,33.201242,-117.296984
1998,KINDRED HOSPITAL FORT WORTH- MANSFIELD,1802 HIGHWAY 157 NORTH,MANSFIELD,TX,76063,32.570740,-97.144837


In [27]:
test_zip='07302'
test_zip

'07302'

In [34]:
for i, row in data.iterrows():
   
    if test_zip == row['zip']:
        lat2=row['latitude']
        long2=row['longitude']

print(f"Latitude: {lat2}")
print(f"Longitude: {long2}")

Latitude: 40.72
Longitude: -74.05


In [35]:
p=pi/180
results=[]
zips = []
lats = []
longs = []
hospitals = []
addresses= []
cities = []
states = []

for i, row in hosp_df.iterrows():
    lat1=row['LAT']
    long1=row['LNG']
    
    hospital=hosp_df['hospital_name'][i]
    print(hospital)
    
    address=hosp_df['address'][i]
    city=hosp_df['city'][i]
    state=hosp_df['state'][i]
    zip_code=row['zip']
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((long2-long1)*p))/2
    result=12742*asin(sqrt(a))
    results.append(result)
    zips.append(zip_code)
    lats.append(lat1)
    longs.append(long1)
    hospitals.append(hospital)
    #print(hospitals)
    addresses.append(address)
    cities.append(city)
    states.append(state)
    #print(f"Zip Code: {zip_code}, Result: {result}")
    

ST VINCENT HOSPITAL
CURAHEALTH PITTSBURGH
MONUMENT HEALTH CUSTER HOSPITAL
ASCENSION ST VINCENT'S SOUTHSIDE
LOURDES HOSPITAL
REGIONAL HOSPITAL OF SCRANTON
WEST SUBURBAN MEDICAL CENTER
ANDERSON COUNTY HOSPITAL
OHIOHEALTH MANSFIELD HOSPITAL
UNITED MEMORIAL MEDICAL CENTER
INTEGRIS Community Hospital - Del City
HOSPITAL SAN CARLOS BORROMEO
DOCTORS HOSPITAL AT RENAISSANCE
COOLEY DICKINSON HOSPITAL INC,THE
STONY BROOK UNIVERSITY HOSPITAL
EAST ORANGE GENERAL HOSPITAL
BENSON HOSPITAL
MORGAN MEDICAL CENTER
TROY REGIONAL MEDICAL CENTER
WESTERN PLAINS MEDICAL COMPLEX
HOSPITAL UNIVERSITARIO DE ADULTO
LANDMARK MEDICAL CENTER
STAFFORD HOSPITAL, LLC
UT HEALTH EAST TEXAS TYLER REGIONAL HOSPITAL
PROVIDENCE MEDFORD MEDICAL CENTER
SIDNEY REGIONAL MEDICAL CENTER
CHI ST JOSEPH HEALTH REGIONAL HOSPITAL
MEDICAL CENTER OF PEACH COUNTY, NAVICENT HEALTH
MERIT HEALTH MADISON
ST MARY MEDICAL CENTER
VANDERBILT WILSON COUNTY HOSPITAL
CLEVELAND CLINIC
ISLAND HOSPITAL
HOLY FAMILY MEMORIAL
UNIVERSITY MEDICAL CENTER-MES

In [36]:
df = pd.DataFrame(list(zip(results, zips, lats, longs, hospitals, addresses, cities, states)),
               columns =['distance', 'zip_codes', 'latitude', 'longitude', 'hospital', 'address', 'city', 'state'])
df

,distance,zip_codes,latitude,longitude,hospital,address,city,state
0,253.899710,01608,42.261980,-71.801462,ST VINCENT HOSPITAL,123 SUMMER STREET,WORCESTER,MA
1,519.482236,15071,40.412105,-80.187799,CURAHEALTH PITTSBURGH,7777 STEUBENVILLE PIKE,OAKDALE,PA
2,2446.303721,57730,43.720822,-103.641848,MONUMENT HEALTH CUSTER HOSPITAL,1220 MONTGOMERY STREET,CUSTER,SD
3,1345.299130,32216,30.277467,-81.582904,ASCENSION ST VINCENT'S SOUTHSIDE,4201 BELFORT RD,JACKSONVILLE,FL
4,1322.080972,42003,37.004848,-88.577941,LOURDES HOSPITAL,1530 LONE OAK ROAD,PADUCAH,KY
...,...,...,...,...,...,...,...,...
1995,3943.912125,90247,33.890853,-118.297967,MEMORIAL HOSPITAL OF GARDENA,1145 W REDONDO BEACH BLVD,GARDENA,CA
1996,740.215256,48060,42.980388,-82.461014,MCLAREN PORT HURON,1221 PINE GROVE AVE,PORT HURON,MI
1997,3891.698732,92056,33.201242,-117.296984,TRI-CITY MEDICAL CENTER,4002 VISTA WAY,OCEANSIDE,CA
1998,2242.056395,76063,32.570740,-97.144837,KINDRED HOSPITAL FORT WORTH- MANSFIELD,1802 HIGHWAY 157 NORTH,MANSFIELD,TX


In [37]:
fifteen_miles = df[df['distance']<= 15]
fifteen_miles

,distance,zip_codes,latitude,longitude,hospital,address,city,state
15,14.703326,07018,40.755897,-74.217961,EAST ORANGE GENERAL HOSPITAL,300 CENTRAL AVE,EAST ORANGE,NJ
68,11.200971,10310,40.632648,-74.116148,RICHMOND UNIVERSITY MEDICAL CENTER,355 BARD AVENUE,STATEN ISLAND,NY
118,8.310200,11215,40.662688,-73.986740,NEW YORK-PRESBYTERIAN BROOKLYN METHODIST HOSPITAL,506 SIXTH STREET,BROOKLYN,NY
120,2.480256,07306,40.734924,-74.071875,CAREPOINT HEALTH-CHRIST HOSPITAL,176 PALISADE AVE,JERSEY CITY,NJ
249,9.438917,10021,40.769258,-73.958751,LENOX HILL HOSPITAL,100 EAST 77TH STREET,NEW YORK,NY
291,6.660957,10016,40.745224,-73.978297,BELLEVUE HOSPITAL CENTER,462 FIRST AVENUE,NEW YORK,NY
461,13.508548,07112,40.709361,-74.209662,NEWARK BETH ISRAEL MEDICAL CENTER,201 LYONS AVE,NEWARK,NJ
468,11.002989,11237,40.704160,-73.921139,WYCKOFF HEIGHTS MEDICAL CENTER,374 STOCKHOLM STREET,BROOKLYN,NY
1354,6.660957,10016,40.745224,-73.978297,NYU LANGONE HOSPITALS,550 FIRST AVENUE,NEW YORK,NY
1681,8.841879,10065,40.764612,-73.963122,NEW YORK-PRESBYTERIAN HOSPITAL,525 EAST 68TH STREET,NEW YORK,NY


In [38]:
#Base Map
usa = [37.090240, -95.712891]
Map = folium.Map(usa, zoom_start=4)
Map

In [39]:
for i in range(0, len(fifteen_miles)):
    folium.Marker([fifteen_miles.iloc[i]["latitude"], fifteen_miles.iloc[i]["longitude"]], popup=fifteen_miles.iloc[i]["hospital"]).add_to(Map)
Map


In [40]:
print(test_zip)

07302


In [45]:
import folium.plugins
from folium.plugins import MarkerCluster
folium.plugins.MarkerCluster()
marker_cluster = MarkerCluster().add_to(Map)
#marker_cluster = folium.MarkerCluster("hopitals").add_to(Map)

# Load locally stored colchester public toilets data

for i in range(0, len(fifteen_miles)):
    folium.Marker([fifteen_miles.iloc[i]["latitude"], fifteen_miles.iloc[i]["longitude"]], popup=fifteen_miles.iloc[i]["hospital"]).add_to(marker_cluster)
Map
 
# my_string = 'date: {}, value: {}'.format(date_occured, crime_level)
# Popup(my_string)

In [48]:
fifteen_miles.iloc[0]["zip_codes"]


'07018'